In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd gdrive/MyDrive/AI/experiment/DrivingHelper/

/content/gdrive/MyDrive/AI/experiment/DrivingHelper


In [3]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from os import listdir

import tensorflow as tf

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

In [ ]:
#### WITTHOUT SETTING NUMBER OF EACH CORNERS ####
#### FINALQ

beginner_path = 'beginner_expert_processedData/beginner/beginner_'
expert_path = 'beginner_expert_processedData/expert/expert_'
left_column = [
#'Time',
    'Distance','GPS Latitude','GPS Longitude','Damper Velocity (Calc) FL','Damper Velocity (Calc) FR','Damper Velocity (Calc) RL',
'Damper Velocity (Calc) RR','Corr Dist','Corr Dist (Unstretched)','Corr Speed','Brake Pos',
'CG Accel Lateral','CG Accel Longitudinal','CG Accel Vertical','CG Height','Camber FL','Camber FR','Camber RL','Camber RR','Car Coord X',
'Car Coord Y','Car Coord Z','Car Pos Norm','Chassis Pitch Angle','Chassis Pitch Rate','Chassis Roll Angle','Chassis Roll Rate',
'Chassis Velocity X','Chassis Velocity Y','Chassis Velocity Z','Chassis Yaw Rate','Drive Train Speed','Engine RPM','Ground Speed',
'Ride Height FL','Ride Height FR','Ride Height RL','Ride Height RR','Road Temp','Self Align Torque FL','Self Align Torque FR',
'Self Align Torque RL','Self Align Torque RR','Session Time Left','Steering Angle','Suspension Travel FL','Suspension Travel FR',
'Suspension Travel RL','Suspension Travel RR','Tire Load FL','Tire Load FR','Tire Load RL','Tire Load RR','Tire Loaded Radius FL',
'Tire Loaded Radius FR','Tire Loaded Radius RL','Tire Loaded Radius RR','Tire Pressure FL','Tire Pressure FR','Tire Pressure RL','Tire Pressure RR',
'Tire Rubber Grip FL','Tire Rubber Grip FR','Tire Rubber Grip RL','Tire Rubber Grip RR','Tire Slip Angle FL','Tire Slip Angle FR',
'Tire Slip Angle RL','Tire Slip Angle RR','Tire Slip Ratio FL','Tire Slip Ratio FR','Tire Slip Ratio RL','Tire Slip Ratio RR',
'Tire Temp Core FL','Tire Temp Core FR','Tire Temp Core RL','Tire Temp Core RR','Tire Temp Inner FL','Tire Temp Inner FR',
'Tire Temp Inner RL','Tire Temp Inner RR','Tire Temp Middle FL','Tire Temp Middle FR','Tire Temp Middle RL',
'Tire Temp Middle RR','Tire Temp Outer FL','Tire Temp Outer FR','Tire Temp Outer RL','Tire Temp Outer RR','Toe In FL',
'Toe In FR','Toe In RL','Toe In RR','Wheel Angular Speed FL','Wheel Angular Speed FR','Wheel Angular Speed RL','Wheel Angular Speed RR',
'CG Distance','Lateral Velocity','Longitudinal Velocity','Lateral Acceleration','Longitudinal Acceleration']

curveList = [[103.9, 209.3], [316.6, 399.6], [425.3, 517.9], [590.5, 756.9], [1048.7, 1110.5], [1212.3, 1437.1]]
curve_num = 0
sequences = list()
final_df = pd.DataFrame()
for i in range(1,36):
    final_df = pd.DataFrame()
    file_path = beginner_path + str(i) + '_new2.csv'
    #print(file_path)
    df = pd.read_csv(file_path, header=0)
    df = df.dropna()
    df = df.astype(float)
    for j in range(0, 6):
      tmp_df = df[(df['Distance'] >= curveList[j][0]) & (df['Distance'] <= curveList[j][1])]
      #print(tmp_df)
      tmp_df = tmp_df.loc[:,left_column]
      #tmp_df['curve_num'] = j
      #final_df = final_df.append(tmp_df, ignore_index=True)
      values = tmp_df.values
      sequences.append(values)

    
for i in range(1,20):
    final_df = pd.DataFrame()
    file_path = expert_path + str(i) + '_new2.csv'
    #print(file_path)
    df = pd.read_csv(file_path, header=0)
    df = df.dropna()
    df = df.astype(float)
    for j in range(0, 6):
      tmp_df = df[(df['Distance'] >= curveList[j][0]) & (df['Distance'] <= curveList[j][1])]
      tmp_df = tmp_df.loc[:,left_column]
      #tmp_df['curve_num'] = j
      values = tmp_df.values
      sequences.append(values)
    

In [ ]:
len(sequences)

324

In [ ]:
#### Split -> Frequency

beginner_path = 'beginner_expert_processedData/beginner/beginner_'
expert_path = 'beginner_expert_processedData/expert/expert_'
left_column = [
#'Time',
    'Distance','GPS Latitude','GPS Longitude','Damper Velocity (Calc) FL','Damper Velocity (Calc) FR','Damper Velocity (Calc) RL',
'Damper Velocity (Calc) RR','Corr Dist','Corr Dist (Unstretched)','Corr Speed','Brake Pos',
'CG Accel Lateral','CG Accel Longitudinal','CG Accel Vertical','CG Height','Camber FL','Camber FR','Camber RL','Camber RR','Car Coord X',
'Car Coord Y','Car Coord Z','Car Pos Norm','Chassis Pitch Angle','Chassis Pitch Rate','Chassis Roll Angle','Chassis Roll Rate',
'Chassis Velocity X','Chassis Velocity Y','Chassis Velocity Z','Chassis Yaw Rate','Drive Train Speed','Engine RPM','Ground Speed',
'Ride Height FL','Ride Height FR','Ride Height RL','Ride Height RR','Road Temp','Self Align Torque FL','Self Align Torque FR',
'Self Align Torque RL','Self Align Torque RR','Session Time Left','Steering Angle','Suspension Travel FL','Suspension Travel FR',
'Suspension Travel RL','Suspension Travel RR','Tire Load FL','Tire Load FR','Tire Load RL','Tire Load RR','Tire Loaded Radius FL',
'Tire Loaded Radius FR','Tire Loaded Radius RL','Tire Loaded Radius RR','Tire Pressure FL','Tire Pressure FR','Tire Pressure RL','Tire Pressure RR',
'Tire Rubber Grip FL','Tire Rubber Grip FR','Tire Rubber Grip RL','Tire Rubber Grip RR','Tire Slip Angle FL','Tire Slip Angle FR',
'Tire Slip Angle RL','Tire Slip Angle RR','Tire Slip Ratio FL','Tire Slip Ratio FR','Tire Slip Ratio RL','Tire Slip Ratio RR',
'Tire Temp Core FL','Tire Temp Core FR','Tire Temp Core RL','Tire Temp Core RR','Tire Temp Inner FL','Tire Temp Inner FR',
'Tire Temp Inner RL','Tire Temp Inner RR','Tire Temp Middle FL','Tire Temp Middle FR','Tire Temp Middle RL',
'Tire Temp Middle RR','Tire Temp Outer FL','Tire Temp Outer FR','Tire Temp Outer RL','Tire Temp Outer RR','Toe In FL',
'Toe In FR','Toe In RL','Toe In RR','Wheel Angular Speed FL','Wheel Angular Speed FR','Wheel Angular Speed RL','Wheel Angular Speed RR',
'CG Distance','Lateral Velocity','Longitudinal Velocity','Lateral Acceleration','Longitudinal Acceleration']

curveList = [[103.9, 209.3], [316.6, 399.6], [425.3, 517.9], [590.5, 756.9], [1048.7, 1110.5], [1212.3, 1437.1]]
curve_num = 0
sequences = list()
final_df = pd.DataFrame()
for i in range(1,36):
    file_path = beginner_path + str(i) + '_new2.csv'
    #print(file_path)
    df = pd.read_csv(file_path, header=0)
    df = df.dropna()
    df = df.astype(float)
    for j in range(0, 6):
      tmp_df = df[(df['Distance'] >= curveList[j][0]) & (df['Distance'] <= curveList[j][1])]
      #print(tmp_df)
      tmp_df = tmp_df.loc[:,left_column]
      tmp_df['curve_num'] = j
      tmp_df.reset_index(inplace = True)
      even = tmp_df.iloc[::2]
      odd = tmp_df.iloc[1::2]
      final_df = final_df.append(even, ignore_index=True)
      final_df = final_df.append(odd, ignore_index=True)
    even_values = even.values
    odd_values = odd.values
    sequences.append(even_values)
    sequences.append(odd_values)
    

    
for i in range(1,20):
    file_path = expert_path + str(i) + '_new2.csv'
    #print(file_path)
    df = pd.read_csv(file_path, header=0)
    df = df.dropna()
    df = df.astype(float)
    for j in range(0, 6):
      tmp_df = df[(df['Distance'] >= curveList[j][0]) & (df['Distance'] <= curveList[j][1])]
      tmp_df = tmp_df.loc[:,left_column]
      tmp_df['curve_num'] = j
      tmp_df.reset_index(inplace = True)
      even = tmp_df.iloc[::2]
      odd = tmp_df.iloc[1::2]
      final_df = final_df.append(even, ignore_index=True)
      final_df = final_df.append(odd, ignore_index=True)
    even_values = even.values
    odd_values = odd.values
    sequences.append(even_values)
    sequences.append(odd_values)
    

In [ ]:
print(len(sequences))

324


In [ ]:
print(len(sequences))
print(sequences[1].shape)

324
(46, 102)


In [ ]:
targets = pd.read_csv('target.csv')
targets = targets.values[:,1]

In [ ]:
targets

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
groups

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 2, 2, 2, 3, 3, 3, 3])

In [ ]:
len_sequences = []
for one_seq in sequences:
    len_sequences.append(len(one_seq))
    
print(len_sequences)
pd.Series(len_sequences).describe()

[72, 46, 492, 111, 28, 125, 71, 57, 214, 98, 27, 148, 78, 49, 318, 105, 26, 151, 91, 43, 311, 88, 27, 146, 68, 50, 429, 115, 31, 129, 69, 54, 102, 105, 31, 147, 75, 61, 73, 112, 30, 145, 84, 58, 77, 95, 30, 131, 76, 50, 124, 94, 34, 145, 79, 51, 94, 108, 30, 146, 97, 59, 121, 127, 41, 154, 102, 68, 218, 140, 36, 150, 101, 50, 833, 135, 37, 194, 89, 48, 695, 99, 33, 156, 63, 69, 110, 97, 36, 183, 74, 59, 90, 86, 29, 136, 80, 60, 80, 90, 28, 142, 69, 48, 329, 111, 31, 121, 78, 52, 73, 97, 25, 494, 88, 53, 80, 78, 27, 139, 82, 52, 89, 79, 27, 141, 75, 59, 73, 95, 28, 130, 80, 55, 83, 82, 27, 148, 78, 57, 82, 79, 25, 149, 82, 54, 79, 84, 26, 133, 120, 72, 89, 94, 30, 141, 68, 46, 348, 117, 39, 138, 71, 65, 90, 93, 29, 157, 75, 53, 163, 551, 38, 115, 66, 57, 193, 1816, 65, 119, 71, 49, 158, 89, 28, 171, 68, 68, 127, 152, 27, 147, 80, 75, 121, 120, 30, 184, 71, 73, 136, 125, 32, 167, 76, 56, 111, 108, 29, 237, 68, 51, 71, 87, 25, 123, 64, 45, 68, 86, 26, 132, 64, 46, 69, 86, 26, 124, 71, 50,

count     324.00000
mean       98.82716
std       128.03819
min        25.00000
25%        51.75000
50%        77.00000
75%       115.50000
max      1816.00000
dtype: float64

In [ ]:
#Padding the sequence with the values in last row to max length
to_pad = 1816
new_seq = []
for one_seq in sequences:
    len_one_seq = len(one_seq)
    last_val = one_seq[-1]
    n = to_pad - len_one_seq
   
    to_concat = np.repeat(one_seq[-1], n).reshape(102, n).transpose()
    new_one_seq = np.concatenate([one_seq, to_concat])
    new_seq.append(new_one_seq)
final_seq = np.stack(new_seq)

#truncate the sequence to length 60
from keras.preprocessing import sequence
seq_len = 25
final_seq=sequence.pad_sequences(final_seq, maxlen=seq_len, padding='post', dtype='float', truncating='post')

In [ ]:
groups = pd.read_csv('group.csv', header=0)
groups = groups.values[:,1]

In [ ]:
len(final_seq)

324

In [ ]:
i=3
int(i/2)

1

In [ ]:
len(final_seq)

108

In [ ]:
train = [final_seq[i] for i in range(len(final_seq)) if (groups[int(i/6)]==1)]
validation = [final_seq[i] for i in range(len(final_seq)) if groups[int(i/6)]==2]
test = [final_seq[i] for i in range(len(final_seq)) if groups[int(i/6)]==3]

train_target = [targets[int(i/6)] for i in range(len(final_seq)) if (groups[int(i/6)]==1)]
validation_target = [targets[int(i/6)] for i in range(len(final_seq)) if groups[int(i/6)]==2]
test_target = [targets[int(i/6)] for i in range(len(final_seq)) if groups[int(i/6)]==3]

train

In [ ]:
train = [final_seq[i] for i in range(len(final_seq)) if (groups[int(i/2)]==1)]
validation = [final_seq[i] for i in range(len(final_seq)) if groups[int(i/2)]==2]
test = [final_seq[i] for i in range(len(final_seq)) if groups[int(i/2)]==3]

train_target = [targets[int(i/2)] for i in range(len(final_seq)) if (groups[int(i/2)]==1)]
validation_target = [targets[int(i/2)] for i in range(len(final_seq)) if groups[int(i/2)]==2]
test_target = [targets[int(i/2)] for i in range(len(final_seq)) if groups[int(i/2)]==3]

train

In [ ]:
train = [final_seq[i] for i in range(len(groups)) if (groups[i]==1)]
validation = [final_seq[i] for i in range(len(groups)) if groups[i]==2]
test = [final_seq[i] for i in range(len(groups)) if groups[i]==3]

train_target = [targets[i] for i in range(len(groups)) if (groups[i]==1)]
validation_target = [targets[i] for i in range(len(groups)) if groups[i]==2]
test_target = [targets[i] for i in range(len(groups)) if groups[i]==3]

train

[array([[1212.7       ,   44.05425462,  -78.68331907, ...,  -10.80305127,
          -14.40406911,    5.        ],
        [1214.7       ,   44.05426388,  -78.68333499, ...,   -7.20203456,
          -21.6061031 ,    5.        ],
        [1216.6       ,   44.05427305,  -78.68335045, ...,  -32.40915408,
          -61.2172906 ,    5.        ],
        ...,
        [1412.        ,   44.05574569,  -78.68386963, ...,  -10.80305127,
           25.20712009,    5.        ],
        [1414.        ,   44.0557635 ,  -78.68386559, ...,  -18.00508525,
           28.80813766,    5.        ],
        [1416.        ,   44.05578014,  -78.68386163, ...,   25.20711981,
           43.21220563,    5.        ]]),
 array([[ 1.21380000e+03,  4.40544472e+01, -7.86836110e+01, ...,
          1.08030513e+01, -3.60101716e+01,  5.00000000e+00],
        [ 1.21530000e+03,  4.40544553e+01, -7.86836224e+01, ...,
         -7.20203427e+00, -6.12172906e+01,  5.00000000e+00],
        [ 1.21680000e+03,  4.40544636e+01, -7.868

In [ ]:
train = np.array(train)
validation = np.array(validation)
test = np.array(test)

train_target = np.array(train_target)
validation_target = np.array(validation_target)
test_target = np.array(test_target)

print(train.shape)
print(train_target.shape)

(204, 25, 102)
(204,)


In [ ]:
!pip install keras-tuner
from kerastuner import HyperModel
from kerastuner.tuners import RandomSearch
from tensorflow import keras

     |████████████████████████████████| 71kB 5.5MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=3126732fe876a9ad97d175818c188b8c3692f753a90d6d78b9fbbe3f68e81e26
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=bc9b62a3e1400e3ae88521990dd33b52d56bb0acbc1996c1b921577ad31faf0c
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [ ]:
from tensorflow.keras.models import Sequential
#alpha?
def build_model(hp):
  model = Sequential()
  model.add(LSTM(units=hp.Int('units', min_value=32, max_value=1024, default=256, step=2), input_shape=(seq_len, 104)))
  model.add(Dropout(rate=hp.Float('dropout', min_value=0.0, max_value=0.5, default=0.25, step=0.05)))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=Adam(
      hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
  ), metrics=['accuracy'])
  return model

tuner=RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=30,
    executions_per_trial=2,
    directory='my_dir',
    project_name='driverLSTM_wolabel'
    #project_name='driverLSTM_mod'
)

callback= tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
tuner.search(train, train_target, epochs=100, validation_data=(validation,validation_target), callbacks=[callback])
tuner.results_summary()

Trial 30 Complete [00h 00m 06s]
val_accuracy: 0.6083333492279053

Best val_accuracy So Far: 0.7416666746139526
Total elapsed time: 00h 04m 00s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in my_dir/driverLSTM_wolabel
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
units: 534
dropout: 0.05
learning_rate: 0.001
Score: 0.7416666746139526
Trial summary
Hyperparameters:
units: 446
dropout: 0.1
learning_rate: 0.001
Score: 0.7416666448116302
Trial summary
Hyperparameters:
units: 726
dropout: 0.2
learning_rate: 0.001
Score: 0.7333333492279053
Trial summary
Hyperparameters:
units: 858
dropout: 0.0
learning_rate: 0.001
Score: 0.7333333194255829
Trial summary
Hyperparameters:
units: 326
dropout: 0.05
learning_rate: 0.001
Score: 0.7166666686534882
Trial summary
Hyperparameters:
units: 954
dropout: 0.4
learning_rate: 0.0001
Score: 0.7000000178813934
Trial summary
Hyperparameters:
units: 1018
dropout: 0.15000000000000002
learning

In [ ]:
#Hyperparameter for the final model (without corner numbers)
model = Sequential()
model.add(LSTM(534, input_shape=(seq_len, 103)))
model.add(Dropout(0.05))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
#Hyperparameter for the final model (with corner numbers)
model = Sequential()
model.add(LSTM(846, input_shape=(seq_len, 103)))
model.add(Dropout(0.35))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model = Sequential()
model.add(LSTM(354, input_shape=(seq_len, 103)))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
adam = Adam(learning_rate=0.001)
chk = ModelCheckpoint('best_model.pkl', monitor='val_acc', save_best_only=True, mode='max', verbose=1)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(train, train_target, epochs=200, batch_size=128, callbacks=[chk], validation_data=(validation,validation_target))

Epoch 1/200
2/2 [==============================] - 2s 368ms/step - loss: 0.7192 - accuracy: 0.5245 - val_loss: 0.6927 - val_accuracy: 0.6167
Epoch 2/200
2/2 [==============================] - 0s 34ms/step - loss: 0.6329 - accuracy: 0.6471 - val_loss: 0.6291 - val_accuracy: 0.5833
Epoch 3/200
2/2 [==============================] - 0s 34ms/step - loss: 0.6007 - accuracy: 0.6422 - val_loss: 0.6025 - val_accuracy: 0.6167
Epoch 4/200
2/2 [==============================] - 0s 32ms/step - loss: 0.5784 - accuracy: 0.6961 - val_loss: 0.5937 - val_accuracy: 0.6667
Epoch 5/200
2/2 [==============================] - 0s 33ms/step - loss: 0.5697 - accuracy: 0.7010 - val_loss: 0.5782 - val_accuracy: 0.7167
Epoch 6/200
2/2 [==============================] - 0s 34ms/step - loss: 0.5517 - accuracy: 0.6961 - val_loss: 0.5792 - val_accuracy: 0.7000
Epoch 7/200
2/2 [==============================] - 0s 37ms/step - loss: 0.5573 - accuracy: 0.7059 - val_loss: 0.5653 - val_accuracy: 0.7167
Epoch 8/200
2/2 [==

In [ ]:
model.evaluate(test,test_target)

2/2 [==============================] - 0s 7ms/step - loss: 0.4191 - accuracy: 0.8500


[0.4191489517688751, 0.8500000238418579]

In [ ]:
adam = Adam(learning_rate=0.001)
chk = ModelCheckpoint('best_model.pkl', monitor='val_acc', save_best_only=True, mode='max', verbose=1)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(train, train_target, epochs=200, batch_size=128, callbacks=[chk], validation_data=(validation,validation_target))

In [ ]:
model.evaluate(test,test_target)

1/1 [==============================] - 0s 18ms/step - loss: 0.3332 - accuracy: 0.9000


[0.33322328329086304, 0.8999999761581421]